### Mid-term for HDS5210

Your supervisor is concerned about 4-year survival risks for COPD. She has asked for you to do some analysis using a new metric, BODE. BODE is an improvement on a previous metric and promises to provide insight on survival risks.

BODE is defined here. https://www.mdcalc.com/calc/3916/bode-index-copd-survival#evidence

Your assignment is to create a BODE calculation, use it to calculate BODE scores and BODE survival rates for a group of patients. Then we want to evaluate the average BODE scores and BODE survival rates for each area hospital.

Your patient input file will have the following columns:
NAME,SSN,LANGUAGE,JOB,HEIGHT_M,WEIGHT_KG,fev_pct,dyspnea_description,distance_in_meters,hospital

BODE calculations require a BMI value, so you will have to create a function for it.

Your output should be in the form of two CSV files, patient_output.csv and hospital_output.csv.

Patient_output will have the following columns:
NAME,BODE_SCORE,BODE_RISK,HOSPITAL

Hospital output will have the following columns:
HOSPITAL_NAME, COPD_COUNT, PCT_OF_COPD_CASES_OVER_BEDS, AVG_SCORE, AVG_RISK

Each function you create should have documentation and a suitable number of test cases. If the input data could be wrong, make sure to raise a Value Error.

For this assignment, use the doctest, json, and csv libraries. Pandas is not allowed for this assignment.

In [1]:
import doctest
import json
import csv

### Step 1: Calculate BMI

In [4]:
def calculate_bmi(height_m, weight_kg):
   """
    Calculate Body Mass Index (BMI) using the formula:

    BMI = weight_kg / (height_m^2)

    Args:
        weight_kg (float): The weight of the patient in kilograms.
        height_m (float): The height of the patient in meters.

    Returns:
        float: The calculated BMI.

    Raises:
        ValueError: If weight or height is non-positive.

    >>> calculate_bmi(70, 1.75)
    22.857142857142858
    >>> calculate_bmi(85, 1.8)
    26.23456790123457
    """
   if weight_kg <= 0 or height_m <= 0:
        raise ValueError("Weight and height must be positive values.")

   return weight_kg / (height_m ** 2)




### Step 2: Calculate BODE Score

In [5]:
 def calculate_bode_score(bmi, fev_pct, dyspnea_description, distance_m):
    """
    Calculate the BODE score based on the following criteria:
    - BMI score
    - FEV% score
    - Dyspnea score
    - Distance walked score

    Args:
        bmi (float): Body Mass Index of the patient.
        fev_pct (float): Percentage of forced expiratory volume.
        dyspnea_description (str): Description of the patient's dyspnea.
        distance_m (float): The distance the patient can walk in 6 minutes (in meters).

    Returns:
        tuple: (BODE score, BODE risk category)

    >>> calculate_bode_score(22, 60, 'Moderate', 400)
    (3, 'Moderate')
    >>> calculate_bode_score(28, 40, 'Severe', 200)
    (9, 'High')
    """

    # BMI scoring
    if bmi < 21:
        bmi_score = 0
    elif 21 <= bmi <= 25:
        bmi_score = 1
    elif 26 <= bmi <= 30:
        bmi_score = 2
    else:
        bmi_score = 3

    # FEV% scoring
    if fev_pct > 80:
        fev_score = 0
    elif 50 <= fev_pct <= 80:
        fev_score = 1
    elif 30 <= fev_pct < 50:
       fev_score = 2
    else:
        fev_score = 3

    # Dyspnea scoring
    dyspnea_mapping = {
        'None': 0,
        'Mild': 1,
        'Moderate': 2,
        'Severe': 3
    }
    dyspnea_score = dyspnea_mapping.get(dyspnea_description, 0)

    # Distance walked scoring
    if distance_m > 400:
        distance_score = 0
    elif 200 <= distance_m <= 400:
        distance_score = 1
    elif 100 <= distance_m < 200:
        distance_score = 2
    else:
        distance_score = 3
    # Calculate total BODE score
    total_bode_score = bmi_score + fev_score + dyspnea_score + distance_score




### Step 3: Calculate BODE Risk

In [10]:
if total_bode_score <= 2:
        risk_category = 'Low'
elif 3 <= total_bode_score <= 5:
        risk_category = 'Moderate'
else:
        risk_category = 'High'
 return total_bode_score, risk_category

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 7)

### Step 4: Load Hospital Data

In [13]:
#Main Function to Process Patients and Hospitals
def process_patients(patient_csv, hospital_json, patient_output_file, hospital_output_file):
    """
    Main function to process patient data, calculate BODE score and risk, and write the output to CSV files.

    Args:
        patient_csv (str): Path to the CSV file containing patient data.
        hospital_json (str): Path to the JSON file containing hospital data.
        patient_output_file (str): Path to the output CSV file for patient data.
        hospital_output_file (str): Path to the output CSV file for hospital data.
    """
    # Load hospital data
    with open(hospital_json, 'r') as file:
        hospitals = json.load(file)

    # Initialize lists for results
    patient_results = []
    hospital_output_list = {hospital['name']: {'count': 0, 'bode_scores': [], 'bode_risks': []}
                            for hospital in hospitals}

    # Read patient data
    with open(patient_csv, 'r') as file:
        reader = csv.DictReader(file)

        for row in reader:
            try:
                # Extract patient data
                name = row['NAME']
                weight_kg = float(row['WEIGHT_KG'])
                height_m = float(row['HEIGHT_M'])
                fev_pct = float(row['fev_pct'])
                dyspnea_description = row['dyspnea_description']
                distance_m = float(row['distance_in_meters'])
                hospital = row['hospital']

                # Calculate BMI
                bmi = calculate_bmi(weight_kg, height_m)

                # Calculate BODE score and risk
                bode_score, bode_risk = calculate_bode_score(bmi, fev_pct, dyspnea_description, distance_m)

                # Store patient result
                patient_results.append([name, bode_score, bode_risk, hospital])

                # Update hospital data
                if hospital in hospital_output_list:
                    hospital_output_list[hospital]['count'] += 1
                    hospital_output_list[hospital]['bode_scores'].append(bode_score)
                    hospital_output_list[hospital]['bode_risks'].append(bode_risk)
            except ValueError as e:
                print(f"Error processing patient {row['NAME']}: {e}")




### Step 5: Main business logic

Call BODE Score, BODE Risk functions for each patient.

For each hospital, calculate Avg BODE score and Avg BODE risk and count the number of cases for each hospital.

In [18]:
patient_csv = "patient.csv"
hospital_json = "hospitals.json"

patient_output_file = "patient_output.csv"
hospital_output_file = "hospital_output.csv"

###Calculate average BODE score and risk for each hospital

 for hospital, data in hospital_output_list.items():
        if data['count'] > 0:
            avg_score = sum(data['bode_scores']) / data['count']
            avg_risk = data['bode_risks'].count('High') / data['count']  # Percentage of high-risk patients
        else:
            avg_score = 0
            avg_risk = 0

        hospital_output_list[hospital] = [
            hospital, data['count'], avg_risk, avg_score
        ]

###
patient_results = []
hospital_output_list = []

#Write Patient_output.csv
with open(patient_output_file, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerows(patient_results)
#Write Hospital_output.csv
with open(hospital_output_file, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerows(hospital_output_list)

NameError: name 'hospital_output_list' is not defined